# 2015년 GSW 의 플레이오프 우승 시점을 기준으로 승률 상승 요인 분석

## 분석 내용
- GSW의 공격 효율성과 수비 효율성이 승률에 얼마나 영향을 미치는지 분석

### 1. GSW의 공격율이 승률에 영향을 미쳤을까?
NBA에서는 공격력을 공격 효율성으로 표현합니다.   

**공격 효율성((Offensive Rating)) 이란?**   
- 100번의 포제션(공격 기회)당 득점 생산력
- 주요 지표로는 TS%(true shooting percentage)와 eFG%(effective field goal percentage)가 사용되는데 이 프로젝트에서는 TS% 지표로 분석
- 공격효율성 = 득점 / 2 x {야투시도 + (0.44 x 자유투시도)}

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 원본 데이터 로드
games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
games_details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")

In [ ]:
# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [ ]:
# games 테이블 전처리
games = games.drop_duplicates(subset=['GAME_ID'], keep='first')
games['HOME_TEAM_WINS'] = games['HOME_TEAM_WINS'].dropna()
games['FG3_PCT_home'] = games['FG3_PCT_home'].fillna(0)
games['PTS_home'] = pd.to_numeric(games['PTS_home'], errors='coerce').fillna(0).astype(int)
games['AST_home'] = pd.to_numeric(games['AST_home'], errors='coerce').fillna(0).astype(int)
games['REB_home'] = pd.to_numeric(games['REB_home'], errors='coerce').fillna(0).astype(int)
games['PTS_away'] = pd.to_numeric(games['PTS_away'], errors='coerce').fillna(0).astype(int)
games['AST_away'] = pd.to_numeric(games['AST_away'], errors='coerce').fillna(0).astype(int)
games['REB_away'] = pd.to_numeric(games['REB_away'], errors='coerce').fillna(0).astype(int)
games['GAME_STATUS_TEXT'] = games['GAME_STATUS_TEXT'].astype('string').fillna('')

games.info()

In [ ]:
# games_details 테이블 전처리
games_details['FG3A'] = pd.to_numeric(games_details['FG3A'], errors='coerce').fillna(0).astype(int)
games_details['FG3M'] = pd.to_numeric(games_details['FG3M'], errors='coerce').fillna(0).astype(int)
games_details['FG3_PCT'] = pd.to_numeric(games_details['FG3_PCT'], errors='coerce').fillna(0)
games_details['COMMENT'] = games_details['COMMENT'].astype('string').fillna('')
games_details['MIN'] = games_details['MIN'].astype('string').fillna('00:00')
games_details['TEAM_ABBREVIATION'] = games_details['TEAM_ABBREVIATION'].astype('string').fillna('')
games_details['TEAM_CITY'] = games_details['TEAM_CITY'].astype('string').fillna('')
games_details['PLAYER_NAME'] = games_details['PLAYER_NAME'].astype('string').fillna('')
games_details['NICKNAME'] = games_details['NICKNAME'].astype('string').fillna('')
games_details['START_POSITION'] = games_details['START_POSITION'].astype('string').fillna('')
games_details['OREB'] =  pd.to_numeric(games_details['OREB'], errors='coerce').fillna(0).astype(int)
games_details['DREB'] =  pd.to_numeric(games_details['DREB'], errors='coerce').fillna(0).astype(int)
games_details['AST'] =  pd.to_numeric(games_details['AST'], errors='coerce').fillna(0).astype(int)
games_details['STL'] =  pd.to_numeric(games_details['STL'], errors='coerce').fillna(0).astype(int)
games_details['BLK'] =  pd.to_numeric(games_details['BLK'], errors='coerce').fillna(0).astype(int)
games_details['PF'] =  pd.to_numeric(games_details['PF'], errors='coerce').fillna(0).astype(int)
games_details['PTS'] =  pd.to_numeric(games_details['PTS'], errors='coerce').fillna(0).astype(int)
games_details['FGM'] =  pd.to_numeric(games_details['FGM'], errors='coerce').fillna(0).astype(int)
games_details['FGA'] =  pd.to_numeric(games_details['FGA'], errors='coerce').fillna(0).astype(int)
games_details['FTM'] =  pd.to_numeric(games_details['FTM'], errors='coerce').fillna(0).astype(int)
games_details['FTA'] =  pd.to_numeric(games_details['FTA'], errors='coerce').fillna(0).astype(int)
games_details['TO'] =  pd.to_numeric(games_details['TO'], errors='coerce').fillna(0).astype(int)
games_details['REB'] =  pd.to_numeric(games_details['REB'], errors='coerce').fillna(0).astype(int)
games_details['PLUS_MINUS'] =  pd.to_numeric(games_details['PLUS_MINUS'], errors='coerce').fillna(0).astype(int)

games_details.info()

In [ ]:
# teams 테이블 전처리
teams['ABBREVIATION'] = teams['ABBREVIATION'].astype('string').str.upper()
teams['NICKNAME'] = teams['NICKNAME'].astype('string')
teams['CITY'] = teams['CITY'].astype('string')
teams['ARENA'] = teams['ARENA'].astype('string')
teams['OWNER'] = teams['OWNER'].astype('string')
teams['HEADCOACH'] = teams['HEADCOACH'].astype('string')
teams['DLEAGUEAFFILIATION'] = teams['DLEAGUEAFFILIATION'].astype('string')
teams['GENERALMANAGER'] = teams['GENERALMANAGER'].astype('string')
teams.info()

# 2003년 부터 2018년까지의 승률 변화

In [ ]:
# gsw team id
gsw_team_id = 1610612744

In [ ]:
# 2018년 이하 시즌만 필터링
# gsw 이긴 홈 경기
home_games = games[(games['HOME_TEAM_ID'] == gsw_team_id) & (games['SEASON'] <= 2018)].copy()
home_games['WIN'] = home_games['HOME_TEAM_WINS']
home_games['TEAM_ID'] = gsw_team_id

# gsw 이긴 원정 경기
away_games = games[(games['VISITOR_TEAM_ID'] == gsw_team_id) & (games['SEASON'] <= 2018)].copy()
away_games['WIN'] = 1 - away_games['HOME_TEAM_WINS']
away_games['TEAM_ID'] = gsw_team_id

# 홈+원정 경기 합치기
gsw_all = pd.concat([home_games, away_games], ignore_index=True)

In [ ]:
# 시즌별 승률 계산
season_stats = gsw_all.groupby('SEASON').agg({
    'WIN': ['sum', 'count', 'mean']
}).reset_index()

# 컬럼 재정의
season_stats.columns = ['SEASON', 'WINS', 'GAMES', 'WIN_PCT']

In [ ]:
# 시즌별 승률 그래프
plt.figure(figsize=(12, 6))
plt.plot(season_stats['SEASON'], season_stats['WIN_PCT'], 
         marker='o', linewidth=2, markersize=8)

# 2015 우승 시즌 기점
if 2015 in season_stats['SEASON'].values:
    plt.axvline(x=2015, color='red', linestyle='--', 
                label='2015 우승 시즌', linewidth=2)

# 모든 시즌 년도 표시
plt.xticks(season_stats['SEASON'])

plt.xlabel('시즌', fontsize=12)
plt.ylabel('승률', fontsize=12)
plt.title('Golden State Warriors 시즌별 승률 추이', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

2015년 GSW의 우승 시점을 기준으로 2012년부터 승률이 오르기 시작하다가 2015년 우승을 했고 2016년쯤 부터 다시 하락하기 시작합니다.   
저는 2015년 우승을 기점으로 2012년부터 승률이 점점 높아지면서 팀의 어떤 부분이 승률에 영향을 주었는지 분석 해보겠습니다.

In [ ]:
merged_all = gsw_all.merge(games_details, on='GAME_ID', how='inner')

basic_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS', 'PLUS_MINUS', 'FGM', 'FGA', 'FG3M', 'FTA']
season_stats = merged_all.groupby(['SEASON'])[basic_stats].mean(numeric_only=True).reset_index()
season_stats

# 공격 효율성(TS%) 계산
season_stats['TS%'] = season_stats['PTS'] / (2 * (season_stats['FGA'] + 0.44 * season_stats['FTA'])) * 100
season_stats[['SEASON', 'TS%']]


# 1. 우리팀의 공격 효율성 상승이 승률에 영향을 미쳤을까?

In [ ]:
# 시즌별 승률 계산 (gsw_all에서)
win_stats = gsw_all.groupby('SEASON').agg({
    'WIN': 'mean'
}).reset_index()
win_stats.columns = ['SEASON', 'WIN_PCT']

# TS%와 승률 병합
corr_data = season_stats[['SEASON', 'TS%']].merge(win_stats, on='SEASON')

# 상관관계 계산
correlation = corr_data['TS%'].corr(corr_data['WIN_PCT'])

# 그래프 그리기
fig, ax1 = plt.subplots(figsize=(12, 6))

# TS% (왼쪽 y축)
color1 = '#FFC72C'
ax1.set_xlabel('시즌', fontsize=12)
ax1.set_ylabel('TS% (공격 효율)', color=color1, fontsize=12)
ax1.plot(corr_data['SEASON'], corr_data['TS%'], color=color1, marker='o', linewidth=2, label='공격 효율성')
ax1.tick_params(axis='y', labelcolor=color1)

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 승률 (오른쪽 y축)
ax2 = ax1.twinx()
color2 = 'tab:blue'
ax2.set_ylabel('승률', color=color2, fontsize=12)
ax2.plot(corr_data['SEASON'], corr_data['WIN_PCT'], color=color2, marker='s', linewidth=2, label='승률')
ax2.tick_params(axis='y', labelcolor=color2)

# 모든 시즌 년도 표시
ax1.set_xticks(corr_data['SEASON'])

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.title(f'GSW 시즌별 TS%와 승률 비교 (상관계수: {correlation:.3f})', fontsize=14, fontweight='bold')
fig.tight_layout()
plt.show()

print(f"TS%와 승률의 상관계수: {correlation:.3f}")

## 상관계수 해석

**TG% (True Shooting %)**는 공격 효율성을 의미합니다.
- 값이 **높을수록** 공격이 좋음
- 값이 **낮을수록** 공격이 나쁨

| 상관계수 | 해석 |
|---------|------|
| 0.7 ~ 1.0 | 강한 양의 상관관계 |
| 0.3 ~ 0.7 | 중간 양의 상관관계 |
| -0.3 ~ 0.3 | 약한 상관관계 |

공격 효율성에 대한 분석 결과를 보면, 상관계수가 0.370으로 승률과 약한 상관관계를 보입니다.   
따라서 우리팀의 공격 효율성이 승률에는 큰 영향을 미치지 않은 것으로 판단됩니다.

# 2. 우리팀의 수비 효율성 상승이 승률에 영향을 미쳤을까?

In [ ]:
# 시즌별 팀 스탯 그룹핑
season_sums = merged_all.groupby('SEASON')[['FGA', 'OREB', 'TO', 'FTA']].sum()

# 시즌별 상대팀 PTS 계산
# GSW 홈 경기 → 상대팀 PTS = PTS_away
# GSW 원정 경기 → 상대팀 PTS = PTS_home
gsw_home = gsw_all[gsw_all['HOME_TEAM_ID'] == gsw_team_id].copy()
gsw_home['opp_PTS'] = gsw_home['PTS_away']

gsw_away = gsw_all[gsw_all['VISITOR_TEAM_ID'] == gsw_team_id].copy()
gsw_away['opp_PTS'] = gsw_away['PTS_home']

opp_pts = pd.concat([gsw_home, gsw_away])
season_sums['opp_PTS'] = opp_pts.groupby('SEASON')['opp_PTS'].sum()

# 포제션 계산: FGA - OREB + TO + (0.4 * FTA)
season_sums['possessions'] = (
    season_sums['FGA']
    - season_sums['OREB']
    + season_sums['TO']
    + (0.4 * season_sums['FTA'])
)

# 수비 효율성 (Defensive Rating) 계산
season_sums['defensive_rating'] = (season_sums['opp_PTS'] / season_sums['possessions']) * 100

season_sums

In [ ]:
# 승률과 수비효율성 상관관계 그래프

# 시즌별 승률 계산
win_stats = gsw_all.groupby('SEASON').agg({'WIN': 'mean'}).reset_index()
win_stats.columns = ['SEASON', 'WIN_PCT']

# defensive_rating과 승률 병합
defense_corr = season_sums.reset_index()[['SEASON', 'defensive_rating']].merge(win_stats, on='SEASON')

# 상관관계 계산
correlation = defense_corr['defensive_rating'].corr(defense_corr['WIN_PCT'])

# 그래프 그리기
fig, ax1 = plt.subplots(figsize=(12, 6))

# Defensive Rating (왼쪽 y축) - 낮을수록 좋음
color1 = '#FFC72C'
ax1.set_xlabel('시즌', fontsize=12)
ax1.set_ylabel('DRTG% (수비 효율성)', color=color1, fontsize=12)
ax1.plot(defense_corr['SEASON'], defense_corr['defensive_rating'], color=color1, marker='o', linewidth=2, label='수비 효율성')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.invert_yaxis()  # 수비 효율성은 낮을수록 좋으므로 y축 반전

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 승률 (오른쪽 y축)
ax2 = ax1.twinx()
color2 = 'tab:blue'
ax2.set_ylabel('승률', color=color2, fontsize=12)
ax2.plot(defense_corr['SEASON'], defense_corr['WIN_PCT'], color=color2, marker='s', linewidth=2, label='승률')
ax2.tick_params(axis='y', labelcolor=color2)

# 모든 시즌 년도 표시
ax1.set_xticks(defense_corr['SEASON'])

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.title(f'GSW 시즌별 수비 효율성과 승률 비교 (상관계수: {correlation:.3f})', fontsize=14, fontweight='bold')
fig.tight_layout()
plt.show()

print(f"수비 효율성과 승률의 상관계수: {correlation:.3f}")

## 상관계수 해석

**Defensive Rating**은 100 포제션당 상대팀 실점을 의미합니다.
- 값이 **낮을수록** 수비가 좋음
- 값이 **높을수록** 수비가 나쁨

| 상관계수 | 해석 |
|---------|------|
| -1.0 ~ -0.7 | 강한 음의 상관관계 |
| -0.7 ~ -0.3 | 중간 음의 상관관계 |
| -0.3 ~ 0.3 | 약한 상관관계 |

수비 효율성과 승률의 상관계수는 -0.603으로 중간 정도의 상관 관계를 보입니다. 수비 효율성은 포제션 100 당 우리팀의 실점이므로 낮을 수록 수비를 잘 했다는 의미 입니다.   
공격 효율성과 수비 효율성을 승률과 비교했을 때, 결과적으로 공격 효율성도 승률과 약한 상관관계를 가지지만 수비 효율성이 더 높은 상관관계를 가지므로 수비 효율성이 팀의 승률에 더 영향을 미쳤다고 볼 수 있습니다.   


# 결론
우리팀의 공격 효율성과 승률도 약한 상관관계가 있었으나, 수비 효율성이 승률과 더 큰 상관관계가 있었습니다.   
따라서, 공격 효율성이 상대적으로 승률에 낮은 영향력을 갖기 때문에 공격 패턴의 다각화가 필요하다고 할 수 있습니다.